In [1]:
'''
【项目03】  知乎数据清洗整理和结论研究

作业要求：
1、数据清洗 - 去除空值
要求：创建函数
提示：fillna方法填充缺失数据，注意inplace参数

2、问题1 知友全国地域分布情况，分析出TOP20
要求：
① 按照地域统计 知友数量、知友密度（知友数量/城市常住人口），不要求创建函数
② 知友数量，知友密度，标准化处理，取值0-100，要求创建函数
③ 通过多系列柱状图，做图表可视化
提示：
① 标准化计算方法 = (X - Xmin) / (Xmax - Xmin)
② 可自行设置图表风格

3、问题2 知友全国地域分布情况，分析出TOP20
要求：
① 按照学校（教育经历字段） 统计粉丝数（‘关注者’）、关注人数（‘关注’），并筛选出粉丝数TOP20的学校，不要求创建函数
② 通过散点图 → 横坐标为关注人数，纵坐标为粉丝数，做图表可视化
③ 散点图中，标记出平均关注人数（x参考线），平均粉丝数（y参考线）
提示：
① 可自行设置图表风格

'''


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings  
warnings.filterwarning('ignore')  # 不需要警告

In [30]:
# 问题1 ：数据清洗 - 去除空值
    # 1. 首先判断一下该环境下的  系统默认路径  并改变到想要运行的系统路径下
    # 2. 读取文件 并 打印前五条数据
    # 3. .fillna(value=0,inplace=True,method='ffill')填充数据(自定义函数清洗)
import os 

data_path= r'C:\Python36\Scripts\练习\项目03知乎数据清洗整理和结论研究_资料'
os.chdir(data_path)

data_raw_zhihu = pd.read_csv('知乎数据_201701.csv',engine='python')
data_raw_people = pd.read_csv('六普常住人口数.csv',engine='python')
# data_raw.head()
# print(data_raw['职业1'].dtype,data_raw['关注'].dtype)

def cleaningFillna(df,args):
    for arg in args:
        if df[arg].dtype == 'object':
            df[arg].fillna(value='缺失值',inplace=True)
        else:
            df[arg].fillna(value=0,inplace=True)
    return df

data_zhihu = cleaningFillna(data_raw_zhihu,data_raw_zhihu.columns)
data_people = data_raw_people


print(data_zhihu.head())
print(data_people.head())

                        _id  关注的收藏夹   关注  关注者  关注的问题  关注的话题  关注的专栏   职业1  \
0  587598f89f11daf90617fb7a      52   17    1     30     58      2  交通仓储   
1  587598f89f11daf90617fb7c      27   73   15     87     26      1  高新科技   
2  587598f89f11daf90617fb7e      72   94    1    112     20      4   缺失值   
3  587598f89f11daf90617fb80     174   84    8    895     30      7    金融   
4  587598f89f11daf90617fb82       3  236   64    119     44     17    金融   

    职业2    回答   提问    收藏       个人简介  居住地  所在行业                       教育经历 职业经历  
0    邮政   0.0  0.0   3.0        缺失值  缺失值    邮政                        缺失值  缺失值  
1   互联网  56.0  4.0  14.0        缺失值   重庆   互联网                     重庆邮电大学  缺失值  
2   缺失值   1.0  0.0  21.0        缺失值  缺失值   缺失值                        缺失值  缺失值  
3    财务   0.0  0.0  22.0        缺失值  缺失值    财务                        缺失值  缺失值  
4  证券投资   6.0  0.0  12.0  无求 心静 魔不生   上海  证券投资  雪城大学（Syracuse University）  缺失值  
     省   地区 结尾        常住人口
0  安徽省  安徽省  省  59500468.0
1  

In [64]:
'''
2、问题1 知友全国地域分布情况，分析出TOP20
要求：
① 按照地域统计 知友数量、知友密度（知友数量/城市常住人口），不要求创建函数
② 知友数量，知友密度，标准化处理，取值0-100，要求创建函数
③ 通过多系列柱状图，做图表可视化
提示：
① 标准化计算方法 = (X - Xmin) / (Xmax - Xmin)
② 可自行设置图表风格
'''

data_zhihu_territory = data_zhihu.groupby('居住地').count()
data_raw_people['city'] = data_raw_people['地区'].str[:-1]   # 获取一个新的字段(主要是把“市”去掉)

data = pd.merge(data_zhihu_territory,data_raw_people,left_index=True,right_on='city',how='inner')[['_id','city','常住人口']]

# 计算人口密度
people_density = data['_id'] / data['常住人口']
data['知友密度'] = people_density
# print(type(people_density))

# 将人口密度进行归一化处理(自定义一个函数)
def normalization(df,*args):
    for arg in args:
        min_x = df[arg].min()
        max_x = df[arg].max()

        df[arg+'_nor'] =((df[arg] - min_x) / (max_x - min_x)) * 100
        
    return df

data_nor = normalization(data,'_id','知友密度')


# data_nor.head()
zy_peo_top20 = data_nor.sort_values('_id_nor',ascending=False)[['city','_id_nor']].iloc[:20]
zy_den_top20 = data_nor.sort_values('知友密度_nor',ascending=False)[['city','知友密度_nor']].iloc[:20]

# print(zy_peo_top20)
# print(zy_den_top20)

In [ ]:
'''
3、问题2 知友全国地域分布情况，分析出TOP20
要求：
① 按照学校（教育经历字段） 统计粉丝数（‘关注者’）、关注人数（‘关注’），并筛选出粉丝数TOP20的学校，不要求创建函数
② 通过散点图 → 横坐标为关注人数，纵坐标为粉丝数，做图表可视化
③ 散点图中，标记出平均关注人数（x参考线），平均粉丝数（y参考线）
提示：
① 可自行设置图表风格
'''

